In [ ]:
# Importing Everything required for running the code
from astropy import table 
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import scipy
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

import pandas as pd

import emcee
import corner

from IPython.display import display, Math

# This package is not precompiled on the CTA computers, so a pdf containing all figures will be submitted
# in addition to all the required files
# Download link: https://astroquery.readthedocs.io/en/latest/ 
from astroquery.vizier import Vizier

# Part 1

In the first portion of this project, you will select a sample of stars in the DESI DD-Payne catalog (Zhang et al. submitted) around Sextans and look for overlapping stars in this sample with Theler et al. (2020).

Skim through Zhang et al. and read in the DESI DD-Payne catalog. What stellar atmospheric parameters and elemental abundances did they measure? How many stars with good measurements are included in this catalog? 

In [ ]:
# Importing the mock DESI data to use for the assignment and converting it into a pandas data frame
# Change the string of sim_data as needed
sim_data = '/home/student8/DESI_EDR_DDPAYNE_MOCK.csv'
sim_table_df = pd.read_csv(sim_data)
print('The total number of stars with good measurements are: ',len(sim_table_df)) 

Look up the following quantities of Sextans in the Local Volume Database: Right Ascension (RA), Declination (Dec), and half-light radius (rh). From the DESI catalog, create a subset of stars that are within 20 half-light radii of the center of Sextans. How many stars are included in this sample?

In [ ]:
# Coordinates of Sextans from the database
Sextans = SkyCoord(153.2628, -1.6133, unit = 'deg')

# We can use separation to find the maximum angle, in rads, allowed for stars to be so that they can still be considered
# in the conical shape produced by the provided radius

# Distange from Earth: 85.9 kpc, half-light radius: 16.5 pc (so 20 of them is 0.33 kpc)
# By trig, arctan(0.33/85.9) is the maximum angle in radians we can have (which is about 0.0038) for stars to be within
# 20 half light radii

data_list = []
for star_index in range(sim_table_df.index.size):
    current_coord = SkyCoord(sim_table_df['TARGET_RA'][star_index], sim_table_df['TARGET_DEC'][star_index], unit = 'deg')
    if Sextans.separation(current_coord).radian <= 0.0038:
        data_list.append([sim_table_df.loc[star_index]])
        
sim_table_2_df = pd.DataFrame(data_list)
# This is a table of all stars within 20 half light radii of sextans

print("The number of stars within 20 half-light radii of Sextans is: ", len(sim_table_2_df))

Using the Astroquery python package, download and combine the data in Tables 2, 6, and 10–12 of Theler et al. (2020) from VizieR. How many stars does this catalog have?

In [ ]:
# Initialize Vizier object for querying.
V = Vizier(
    row_limit=-1,
    columns=['**', '_RAJ2000', '_DEJ2000']  # '**' means all 
)

# Query Catalog using the catalog identifier
tbl_list = V.get_catalogs('J/A+A/642/A176')

# Select relevant tables and convert them from astropy tables to pandas dataframes.
table2 = tbl_list[0].to_pandas()
table6 = tbl_list[4].to_pandas()
table10 = tbl_list[6].to_pandas()
table11 = tbl_list[7].to_pandas()
table12 = tbl_list[8].to_pandas()

# Re-index the dataframes with the provided IDs from the catalog
table2.index = table2['ID']
table6.index = table6['ID']
table10.index = table10['ID']
table11.index = table11['ID']
table12.index = table12['ID']

# Combine the 5 tables based on their indices
theler2020_df = pd.concat([table2, table6, table10, table11, table12], axis=1)

# Remove duplicate columns
theler2020_df = theler2020_df.loc[:,~theler2020_df.columns.duplicated()].copy()

print('The number of stars in this catalogue is: ', len(theler2020_df))

Using the Astropy python package, cross-match the two catalogs. Justify your separation criteria. How many stars are in both catalogs?

In [ ]:
# Convert to tables from pandas to cross_match
theler2020 = Table.from_pandas(theler2020_df)
sim_table = Table.from_pandas(sim_table_df)

c = SkyCoord(ra=sim_table['TARGET_RA']*u.degree, dec=sim_table['TARGET_DEC']*u.degree)
catalog = SkyCoord(ra=theler2020['_RAJ2000']*u.degree, dec=theler2020['_DEJ2000']*u.degree)

match, d2d, d3d = c.match_to_catalog_sky(catalog)

# Here the separation criteria is employed
# Due to extremely low number of stars in common, a high separation criteria of 1 is chosen to complete further analysis of
# the data. However, any data from beterrn 0.3 to 1 is acceptable and a lower number is typically preferred.
theler_inDESI = theler2020[match][d2d < 1*u.arcsec]
desi_intheler = sim_table[d2d < 1*u.arcsec]
desi_notintheler = sim_table[d2d >= 1*u.arcsec]

gc_comb = table.hstack((theler_inDESI,desi_intheler))

Visualize your final sample by creating a figure illustrating the spatial distribution (RA vs. Dec) of each catalog. Choose marker shapes and colors that make it easy to see which stars come from each catalog and which stars are included in both.

In [ ]:
# Modifying the tables to just contain RA and DEC for plotting
theler2020_modified = theler2020['_RAJ2000', '_DEJ2000']
sim_table_modified = sim_table['TARGET_RA','TARGET_DEC']
gc_comb_modified = gc_comb['TARGET_RA','TARGET_DEC']

plt.figure(figsize=(10,6))
plt.scatter(sim_table_modified['TARGET_DEC'], sim_table_modified['TARGET_RA'], label = 'Mock DESI Stars', color = 'navy', marker = 'o', s = 3)
plt.scatter(theler2020_modified['_DEJ2000'], theler2020_modified['_RAJ2000'], label = 'Theler2020 Stars', color = 'red', marker = 's', s = 3)
plt.scatter(gc_comb_modified['TARGET_DEC'], gc_comb_modified['TARGET_RA'], label = 'Stars In Both', color = 'yellow', marker = '*', s = 30)
plt.legend(loc="upper left")
plt.xlabel('Declination (deg)')
plt.ylabel('Right Ascension (deg)')
plt.title('Spatial Distribution of Mock DESI Data & Theler2020 Data')
plt.show()



# Part 2

In the second portion of this project, you will perform a preliminary validation and calibration of the DESI DD-Payne catalog to the measurements from high-resolution spectroscopy by Theler et al. (2020).

For each stellar parameter and elemental abundance that the two catalogs have in common, plot the differences between the two catalogs for the cross-matched sample using [Fe/H] from the Theler et al. catalog as the x-axis. Be sure to include appropriate error bars. Add a horizontal line at a value of zero for reference. At first glance, how does the agreement between the catalogs look for each quantity? Is there an obvious offset? A linear trend? Large scatter?

In [ ]:
# We can see that the chemical abundances they have in common are: FeH, MgFe, CaFe, TiFe, CrFe MnFe, NIFe, and their 
# respective errors
# Theler2020 quantities: FeH, MgFe, CaFe, ScFe, TiIFe, TiIIFe, CrFe, MnFe, CoFe, NIFe, BaFe, EuFe
# DESI quantities: FeH, CFe, NFe, MgFe, AlFe, SiFe, CaFe, TiFe, CrFe, MnFe, NIFe
# They do both have Ti however one specifies between TiI and TiII, we omit Ti for this reason

# Creating a new table that will contain the differences and combined error of both sets of data
differences = Table(names = ('RA', 'DEC', 'FeH', 'FeH_err', 'MgFe', 'MgFe_err', 'CaFe', 'CaFe_err',\
                    'CrFe', 'CrFe_err', 'MnFe', 'MnFe_err', 'NIFe', 'NIFe_err'))

# if you subtract two quantities, A and B with estimated errors eA and eB, the result will be 
# A – B with an estimated absolute error of eA + eB.

for i in range(len(theler_inDESI)):
    differences.add_row((theler_inDESI['_RAJ2000'][i], theler_inDESI['_DEJ2000'][i], abs(theler_inDESI['__Fe_H_'][i]) - abs(desi_intheler['FEH'][i]),\
                         theler_inDESI['er__Fe_H_'][i] + desi_intheler['FEH_ERR'][i], abs(theler_inDESI['__Mg_Fe_'][i]) - abs(desi_intheler['MG_FE'][i]),\
                         theler_inDESI['er__Mg_Fe_'][i] + desi_intheler['MG_FE_ERR'][i], abs(theler_inDESI['__Ca_Fe_'][i]) - abs(desi_intheler['CA_FE'][i]),\
                         theler_inDESI['er__Ca_Fe_'][i] + desi_intheler['CA_FE_ERR'][i], abs(theler_inDESI['__Cr_Fe_'][i]) - abs(desi_intheler['CR_FE'][i]),\
                         theler_inDESI['er__Cr_Fe_'][i] + desi_intheler['CR_FE_ERR'][i], abs(theler_inDESI['__Mn_Fe_'][i]) - abs(desi_intheler['MN_FE'][i]),\
                         theler_inDESI['er__Mn_Fe_'][i] + desi_intheler['MN_FE_ERR'][i], abs(theler_inDESI['__NI_Fe_'][i]) - abs(desi_intheler['NI_FE'][i]),\
                         theler_inDESI['er__NI_Fe_'][i] + desi_intheler['NI_FE_ERR'][i],))
    
plt.scatter(theler_inDESI['__Fe_H_'], differences['FeH'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['FeH'], differences['FeH_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Fe/H] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Fe/H] Between Catalogues Compared to Star Metallicity')
plt.show()

plt.scatter(theler_inDESI['__Fe_H_'], differences['MgFe'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['MgFe'], differences['MgFe_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Mg/Fe]Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Mg/Fe] Between Catalogues Compared to Star Metallicity')
plt.show()

plt.scatter(theler_inDESI['__Fe_H_'], differences['CaFe'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['CaFe'], differences['CaFe_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Ca/Fe] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Ca/Fe] Between Catalogues Compared to Star Metallicity')
plt.show()

plt.scatter(theler_inDESI['__Fe_H_'], differences['CrFe'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['CrFe'], differences['CrFe_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Cr/Fe] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Cr/Fe] Between Catalogues Compared to Star Metallicity')
plt.show()

plt.scatter(theler_inDESI['__Fe_H_'], differences['MnFe'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['MnFe'], differences['MnFe_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Mn/Fe] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Mn/Fe] Between Catalogues Compared to Star Metallicity')
plt.show()

plt.scatter(theler_inDESI['__Fe_H_'], differences['NIFe'])
plt.errorbar(theler_inDESI['__Fe_H_'], differences['NIFe'], differences['NIFe_err'], linestyle='None')
plt.axhline(y=0, color='grey', linestyle='-')
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Ni/Fe] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Ni/Fe] Between Catalogues Compared to Star Metallicity')
plt.show()




In [ ]:
# We already have our data, the table differences
# We will examine the differences in Fe/H since that seems to have the most interesting result however this can be performed
# on any of the values above

# It seems one of the values is masked, we will remove this value

x = theler_inDESI['__Fe_H_']
y = differences['FeH']
yerr = differences['FeH_err']

x = []
y = []
yerr = []
for entry in range(len(theler_inDESI['__Fe_H_'])):
    if theler_inDESI['__Fe_H_'][entry] is not np.ma.masked:
        x.append(theler_inDESI['__Fe_H_'][entry])
        
    if differences['FeH'][entry] is not np.ma.masked and differences['FeH'][entry] != 0.0 :
        y.append(differences['FeH'][entry])
        
    if differences['FeH_err'][entry] is not np.ma.masked and differences['FeH_err'][entry] != 0.0:
        yerr.append(differences['FeH_err'][entry])
    
x = np.asarray(x)
y = np.asarray(y)
yerr = np.asarray(yerr)

A = np.vander(x, 2)
C = np.diag(yerr * yerr)
ATA = np.dot(A.T, A / (yerr**2)[:, None])
cov = np.linalg.inv(ATA)
w = np.linalg.solve(ATA, np.dot(A.T, y / yerr**2))
print("Least-squares estimates:")
print("m = {0:.3f} ± {1:.3f}".format(w[0], np.sqrt(cov[0, 0])))
print("b = {0:.3f} ± {1:.3f}".format(w[1], np.sqrt(cov[1, 1])))

def log_likelihood(theta, x, y, yerr):
    m, b, log_f = theta
    model = m * x + b
    # Removing the scatter caused by some function of x and replaing it with a constant
    sigma2 = yerr**2 + 0.5
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

nll = lambda *args: -log_likelihood(*args)
# Initial is chosen based on an estimation by glancing at t
initial = np.array([-0.8, -1, 0.5])
soln = minimize(nll, initial, args=(x, y, yerr))
m_ml, b_ml, log_f_ml = soln.x

print("Maximum likelihood estimates:")
print("m = {0:.3f}".format(m_ml))
print("b = {0:.3f}".format(b_ml))
print("f = {0:.3f}".format(np.exp(log_f_ml)))

def log_prior(theta):
    m, b, log_f = theta
    if -2 < m < 2 and -2 < b < 2 and -10.0 < log_f < 10:
        return 0.0
    return -np.inf
# The priors are based on simple observation. It is intuitive that the slope and intercept would not be too extreme for the
# Errors, so we have chosen these values. We will also weight all of the variables as equal, so we can return 0

def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

pos = soln.x + 1e-4 * np.random.randn(32, 3)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(x, y, yerr)
)
sampler.run_mcmc(pos, 5000, progress=True);

fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["m", "b", "log(f)"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
tau = sampler.get_autocorr_time()
print(tau)

flat_samples = sampler.get_chain(discard=50, thin=15, flat=True)
print(flat_samples.shape)

# Plotting with the MLE as that is our best guess so far
fig = corner.corner(
    flat_samples, labels=labels, truths=[-0.8, -1, np.log(1.694)]
);


In [ ]:
x0 = np.linspace(-4,-1.5,500)
inds = np.random.randint(len(flat_samples), size=100)
for ind in inds:
    sample = flat_samples[ind]
    plt.plot(x0, np.dot(np.vander(x0, 2), sample[:2]), "C1", alpha=0.1)
    
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))   
    
plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
plt.plot(x0, -0.8 * x0 + -1, color = 'red', label="MLE")
plt.plot(x0, -0.153 * x0 + 0.179, color = 'blue', label="LSE")
plt.plot(x0, -0.51 * x0 + -0.502, color = 'green', label="MCMC")
plt.legend(fontsize=14)
plt.xlim(-4, -1.5)
plt.xlabel('[Fe/H] Measurements From Theler2020')
plt.ylabel('[Fe/H] Measurement Differences\n Between Catalogues')
plt.title('Differences in Measured [Fe/H] Between Catalogues Compared to Star Metallicity');